In [30]:
import os
import time
import torch
import torch as tc
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import matplotlib.pyplot as plt
#get_ipython().magic('matplotlib inline')
import torch.nn.functional as F

from PIL import Image
import numpy as np
import sys,os
import math
import gc

In [359]:
nc = 3
ngpu = 1
nz = 100
ngf = 32
ndf = 32
n_extra_d = 0
n_extra_g = 1 # Aqui a jogada é que o gerador deve ser mais poderoso q o detetive
imageSize = 64

In [100]:
n_classes = 10

In [449]:
class _netD_1(nn.Module):
    def __init__(self, ngpu, nz, nc, ndf,  n_extra_layers_d, n_classes):
        super(_netD_1, self).__init__()
        self.ngpu = ngpu
        self.conv1 = nn.Conv2d(in_channels = nc, out_channels = ndf, kernel_size=4, stride=2, padding=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels = ndf, out_channels = ndf*2, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch2 = nn.BatchNorm2d(ndf * 2)
        self.conv3 = nn.Conv2d(in_channels = ndf*2, out_channels = ndf*4, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch3 = nn.BatchNorm2d(ndf * 4)
        self.conv4 = nn.Conv2d(in_channels = ndf*4, out_channels = ndf*8, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch4 = nn.BatchNorm2d(ndf * 8)
        
        self.final_conv = nn.Conv2d(in_channels=ndf*8, out_channels=n_classes,kernel_size=2,stride=1,padding=0,bias=False)
        
    def forward(self, x):
        print('0',x.size())

        x = F.leaky_relu(self.conv1(x),0.02,inplace=True)
        print('1',x.size())
        x = F.leaky_relu(self.batch2(self.conv2(x)),0.02,inplace=True)
        print('2',x.size())
        x = F.leaky_relu(self.batch3(self.conv3(x)),0.02,inplace=True)
        print('3',x.size())
        x = F.leaky_relu(self.batch4(self.conv4(x)),0.02,inplace=True)   
        print('antes final',x.size())
        x = self.final_conv(x)
        print('depois final',x.size())
        
        x = F.sigmoid(x)

        return(x)
        
  

In [454]:
class _netG_1(nn.Module):
    def __init__(self, ngpu, nz, nc , ngf, n_extra_layers_g):
        super(_netG_1, self).__init__()
        self.ngpu = ngpu
        
        self.convt1 = nn.ConvTranspose2d(in_channels = nz, out_channels = ngf * 8, kernel_size=4, stride=2, padding=0, bias=False)
        self.batch1 = nn.BatchNorm2d(ngf*8)
        self.convt2 = nn.ConvTranspose2d(in_channels = ngf * 8, out_channels = ngf * 4, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch2 = nn.BatchNorm2d(ngf*4)
        self.convt3 = nn.ConvTranspose2d(in_channels = ngf * 4, out_channels = ngf * 2, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch3 = nn.BatchNorm2d(ngf*2)
        self.convt4 = nn.ConvTranspose2d(in_channels = ngf*2, out_channels = ngf, kernel_size=4, stride=2, padding=1, bias=False)
        self.batch4 = nn.BatchNorm2d(ngf)
        
        self.final_conv = nn.ConvTranspose2d(in_channels=ngf, out_channels=nc,kernel_size=1,stride=1,padding=0,bias=False)
    def forward(self, x):
        x = F.leaky_relu(self.batch1(self.convt1(x)),0.02,inplace=True)
        x = F.leaky_relu(self.batch2(self.convt2(x)),0.02,inplace=True)
        x = F.leaky_relu(self.batch3(self.convt3(x)),0.02,inplace=True)
        x = F.leaky_relu(self.batch4(self.convt4(x)),0.02,inplace=True)
        x = self.final_conv(x)
        x = F.tanh(x)
        
        return (x)

In [455]:
netG = _netG_1(ngpu, nz, nc, ndf, n_extra_d)


In [456]:
a = tc.randn(64,100,1,1)

In [457]:
output = netG(Variable(a))

In [458]:
print(output.size())

torch.Size([64, 3, 32, 32])


In [450]:

netD = _netD_1(ngpu, nz, nc, ndf, n_extra_d,n_classes=10)
print()


In [451]:
image = tc.randn(64,3,32,32)
print(type(image))
print(image.size())

<class 'torch.FloatTensor'>
torch.Size([64, 3, 32, 32])


In [452]:
output = netD(Variable(image))
print(output.size())

0 torch.Size([64, 3, 32, 32])
1 torch.Size([64, 32, 16, 16])
2 torch.Size([64, 64, 8, 8])
3 torch.Size([64, 128, 4, 4])
antes final torch.Size([64, 256, 2, 2])
depois final torch.Size([64, 10, 1, 1])
torch.Size([64, 10, 1, 1])


In [ ]:
class Net(nn.Module):
    def __init__(self, output, image_size=(1,96,96)):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(image_size[0], 64, kernel_size=3)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3)
        self.conv4 = nn.Conv2d(256, 256, kernel_size=3)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3)
        
        feature_size = self._get_conv_output(image_size)
        
        self.dense1 = nn.Linear(feature_size, 256)
        self.dense2 = nn.Linear(256,128)
        self.dense3 = nn.Linear(128,64)
        self.dense4 = nn.Linear(64, output)
        
    def _get_conv_output(self, shape):
        bs = 1
        input = Variable(torch.rand(bs, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(bs, -1).size(1)
        return n_size

    def _forward_features(self, x):
        x = F.relu(F.dropout(self.conv1(x), training=self.training, p=0.25))
        x = F.relu(F.dropout(F.max_pool2d(self.conv2(x), 2), training=self.training, p=0.25))
        x = F.relu(F.dropout(self.conv3(x), training=self.training, p=0.25))
        x = F.relu(F.dropout(F.max_pool2d(self.conv4(x), 2), training=self.training, p=0.25))
        x = F.relu(F.dropout(F.max_pool2d(self.conv5(x), 2), training=self.training, p=0.25))
        return x
    
    def forward(self, x):
        x = self._forward_features(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.dense1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.dense2(x))
        x = F.relu(self.dense3(x))
        x = F.relu(self.dense4(x))
        
        return x

## Aprendendo a fazer one-hot-encoding

In [159]:
y = torch.LongTensor([1,2,3,4,5,6,7,9])
print(y)
y_onehot = torch.FloatTensor(10, 10)
print(y_onehot.size())


 1
 2
 3
 4
 5
 6
 7
 9
[torch.LongTensor of size 8]

torch.Size([10, 10])


In [160]:
print(y_onehot.zero_())
print(y_onehot.scatter_(1, y, 1))



    0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     0
[torch.FloatTensor of size 10x10]



RuntimeError: invalid argument 3: Index tensor must have same dimensions as output tensor at /opt/conda/conda-bld/pytorch_1503968623488/work/torch/lib/TH/generic/THTensorMath.c:504

In [161]:
import torch

batch_size = 5
nb_digits = 10
# Dummy input that HAS to be 2D for the scatter (you can use view(-1,1) if needed)
y = torch.LongTensor(batch_size,1).random_() % nb_digits
# One hot encoding buffer that you create out of the loop and just keep reusing
y_onehot = torch.FloatTensor(batch_size, nb_digits)

# In your for loop
y_onehot.zero_()
y_onehot.scatter_(1, y, 1)

print(y)
print(y_onehot)


 0
 0
 1
 5
 2
[torch.LongTensor of size 5x1]


    1     0     0     0     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     1     0     0     0     0     0     0     0     0
    0     0     0     0     0     1     0     0     0     0
    0     0     1     0     0     0     0     0     0     0
[torch.FloatTensor of size 5x10]



In [ ]:
a = tc.randn(10,2)
a = Variable(a)

In [163]:
a.f
print(a.size())

torch.Size([10, 2])


In [181]:
# your tensor of 16 x 28 dimensions, 
# where each element has some index (0 to n)
inp = torch.LongTensor(64, 10) % n_classes    
print(inp.size())
print(inp.min())
print(inp.max())

torch.Size([64, 10])
0
4


In [185]:
inp = torch.LongTensor(64, 10) % n_classes    
inp_ = torch.unsqueeze(inp, 2)
print(inp_.size())

torch.Size([64, 10, 1])


In [186]:
#loop
one_hot = torch.FloatTensor(64, 10, n_classes).zero_()
print(one_hot.size())
one_hot.scatter_(2, inp_, 1)

torch.Size([64, 10, 10])



(0 ,.,.) = 
   0   0   1  ...    0   0   0
   1   0   0  ...    0   0   0
   1   0   0  ...    0   0   0
     ...       ⋱       ...    
   0   0   1  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   1   0

(1 ,.,.) = 
   1   0   0  ...    0   0   0
   1   0   0  ...    0   0   0
   1   0   0  ...    0   0   0
     ...       ⋱       ...    
   0   0   0  ...    0   0   0
   1   0   0  ...    0   0   0
   0   0   0  ...    0   1   0

(2 ,.,.) = 
   0   0   0  ...    0   0   0
   1   0   0  ...    0   0   0
   1   0   0  ...    0   0   0
     ...       ⋱       ...    
   1   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
...

(61,.,.) = 
   1   0   0  ...    0   0   0
   0   0   0  ...    0   0   0
   1   0   0  ...    0   0   0
     ...       ⋱       ...    
   1   0   0  ...    0   0   0
   1   0   0  ...    0   0   0
   0   0   0  ...    0   1   0

(62,.,.) = 
   1   0   0  ...    0   0   0
   1   0   0  ...    0   0   0
 

In [198]:
random = tc.randn(64)
print(random.size())
random_hot = random % 10
print(random_hot.size())

torch.Size([64])
torch.Size([64])


In [195]:
inp = torch.LongTensor(16) % 10    
inp_ = torch.unsqueeze(inp, 1)

print(inp.size())
print(inp_.size())
print(inp[0])
print(inp_[0])

torch.Size([16])
torch.Size([16, 1])
2

 2
[torch.LongTensor of size 1]



In [227]:
one_hot = tc.LongTensor(64,10,1) % 10

In [228]:
a = tc.randn(10)
test = torch.unsqueeze(a, 1)
print(test.size())


torch.Size([10, 1])


In [229]:
one_hot = one_hot.zero_()

In [233]:
one_hot_oi = one_hot.scatter_(10,one_hot, 1)
print(one_hot_oi.size())

RuntimeError: dimension out of range (expected to be in range of [-3, 2], but got 10)

In [231]:
print(one_hot[0])
print(one_hot_oi[0])


    1
    0
    0
    0
    0
    0
    0
    0
    0
    0
[torch.LongTensor of size 10x1]


    1
    0
    0
    0
    0
    0
    0
    0
    0
    0
[torch.LongTensor of size 10x1]



In [197]:
one_hot = torch.FloatTensor(16, 10).zero_()
one_hot.scatter_(1, inp_, 1)

print(one_hot.size())

torch.Size([16])
torch.Size([16, 10])


In [192]:
print(inp[0] , one_hot[0])


 4
 0
 6
 0
 4
 0
 8
 8
 0
 0
 4
 0
 8
 0
 8
 8
 0
 0
 0
 6
 8
 0
 8
 8
 0
 6
 0
 0
[torch.LongTensor of size 28]
 
    0     0     0     0     1     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     1     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     0     0     0     1     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     1     0
    0     0     0     0     0     0     0     0     1     0
    1     0     0     0     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     0     0     0     1     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     1     0
    1     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0  

In [236]:
a = tc.randn(10)
b = a.view(-1,10)
print(b.size())
print(b)

torch.Size([1, 10])

-1.6505  0.5080 -0.0269 -0.4995  2.4222 -0.8577 -1.2725  1.3640 -0.2171 -0.6493
[torch.FloatTensor of size 1x10]



In [247]:
a = tc.LongTensor(64) % 10
print(a.size())
a_ = tc.unsqueeze(a,1)
print(a_.size())

torch.Size([64])
torch.Size([64, 1])


In [252]:
one_hot = torch.FloatTensor(64, 10).zero_()
one_hot.scatter_(1, a_, 1)



    0     0     0     0     0     0     1     0     0     0
    0     0     0     0     0     0     1     0     0     0
    0     0     0     0     0     0     0     0     1     0
    0     0     0     0     0     1     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     1     0     0     0     0
    0     0     0     0     0     0     0     0     1     0
    1     0     0     0     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     0     0     0     0     0     0     0     0     1
    1     0     0     0     0     0     0     0     0     0
    1     0     0     0     0     0     0     0     0     0
    0     0     1     0     0     0    

In [ ]:
inp = torch.LongTensor(16, 28) % n    
inp_ = torch.unsqueeze(inp, 2)

one_hot = torch.FloatTensor(16, 28, n).zero_()
one_hot.scatter_(2, inp_, 1)

print(inp)
print(one_hot)

In [264]:
a = tc.randn(10)
a = Variable(a)
b = tc.randn(10)
b = Variable(b)

In [265]:
print(a)
print(b)

Variable containing:
-0.0671
 1.1891
 1.3497
-0.6009
 1.0966
 2.1872
 0.1390
-1.2602
-0.9997
 0.2749
[torch.FloatTensor of size 10]

Variable containing:
 0.7839
 0.8463
 1.6752
-1.2068
-0.4864
-0.7506
-1.8374
 2.0915
 0.1442
-0.5733
[torch.FloatTensor of size 10]



In [266]:
a = a.data.copy_(b)
print(a)


RuntimeError: copy from Variable to torch.FloatTensor isn't implemented

In [267]:
print(type(a))

<class 'torch.autograd.variable.Variable'>
